In [1]:
# temporary solution to crashing
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [2]:
import torch
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import mysql.connector as connector
from pathlib import Path
import wandb
import tqdm

import logging

In [3]:
home = os.path.expanduser('~')
os.chdir(home) # b/c we will be using universal paths

host = '127.0.0.1'
user = 'root' # change to your username
password = 'vasya1' # change to your password
database = 'ai_proj_2025' # for the sake of sanity, we should all have this as the db name 

try:
    conn = connector.connect(
        host = host, 
        user = user, 
        password = password, 
        database = database
    )
    print('success')
except connector.Error as err:
    print(err)

success


Note: the entire CrisisMMD dataset in SQL contains a little over 18,000 images. 

In [4]:
# research_dir = Path(home, 'Desktop', 'Education', 'Spring 2025', 'AI', 'research')
research_dir = Path(home, "ai_research_proj_spring_2025", "research_ai_class_spring_2025") # in lab 409 computer for Agafia
os.chdir(research_dir)

from data_helper import SQLDataset_Informative

os.chdir(home)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\agafi\\ai_research_proj_spring_2025\\research_ai_class_spring_2025'

In [5]:
#Define the convnet (Eventually replae with mobile 2)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3) #specify the size (outer numbers) and amount (middle number) of filters
        self.pool = nn.MaxPool2d(2, 2) #specify pool size first number is size of pool, second is step size
        self.conv2 = nn.Conv2d(16, 8, 3) #new depth is amount of filters in previous conv layer
        self.fc1 = nn.Linear(54*54*8, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 2) #finial fc layer needs 19 outputs because we have 19 layers # ???

    def forward(self, x):

        x = F.relu(self.conv1(x))
     
        x = self.pool(x)
       
        x = F.relu(self.conv2(x))

        x = self.pool(x)

        x = x.view(-1, 54*54*8) # flatten

        x = F.relu(self.fc1(x))    #fully connected, relu         
        x = F.relu(self.fc2(x))    
       
        x = self.fc3(x)     #output    
        return x

In [6]:
from torchvision.transforms import v2

# transforms
transformations = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), antialias=True), 
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
# create train, val, and test sets

data_dir=Path('OneDrive - Stephen F. Austin State University', 'CrisisMMD_v2.0','CrisisMMD_v2.0')

train_set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=data_dir, is_train=True)
val_set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=data_dir, is_val=True)
test_set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=data_dir, is_test=True)

NameError: name 'SQLDataset_Informative' is not defined

In [ ]:
train_set.__len__()

16274

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, batch_size=64)
val_loader = DataLoader(val_set, batch_size=64)
test_loader = DataLoader(test_set, batch_size=64)


In [8]:
model = ConvNet()

In [9]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

2808470

In [13]:
params = [param for param in model.parameters()]
names = [name for name in model.modules()]

for name, param in zip(names, params):
    print(name)
    print(param)
    print('-'*30)

ConvNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=23328, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=2, bias=True)
)
Parameter containing:
tensor([[[[-1.7090e-02, -1.7716e-02, -1.4424e-01],
          [-9.1638e-03, -1.5260e-01,  5.3528e-03],
          [ 1.7386e-01, -8.5570e-02, -1.5085e-01]],

         [[-1.6821e-01,  1.2913e-01,  5.8813e-02],
          [-1.7426e-01,  7.1126e-03, -1.0651e-01],
          [-1.0754e-01, -2.4119e-02,  1.2537e-01]],

         [[ 1.9071e-01, -9.2676e-02, -2.2592e-02],
          [ 1.5451e-01, -1.0522e-01, -7.2909e-02],
          [ 1.0914e-01,  6.3291e-02,  1.7715e-01]]],


        [[[ 4.2871e-02, -3.0366e-02,  4.0613e-02],
          [-1.2580e-01,  5.4030e-02, -1.8546e-01],
         

In [11]:
test_result = model.forward(train_set.__getitem__(0)['image'])
test_result

tensor([[-0.0083,  0.0859]], grad_fn=<AddmmBackward0>)

In [12]:
# importing accuracy metric functions
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
# validation fn

def dev(model, val_loader):
    model.to(device)
    batch_size = val_loader.batch_size
    avg = 'macro' # used when computing certain accuracy metrics
    model.eval()

    eval_loss = 0

    all_preds = []
    all_trues = []

    with torch.no_grad():
        for b, batch in tqdm.tqdm(enumerate(val_loader), 
                             total= len(val_loader), desc=f"Processing validation data"):
            images = batch['image'].to(device)
            labels = batch['label'].to(device)

            raw_logits = model.forward(images)

            preds = torch.argmax(raw_logits, dim=1) # https://discuss.pytorch.org/t/cross-entropy-loss-get-predicted-class/58215

            loss = nn.CrossEntropyLoss()(raw_logits, labels)

            eval_loss += loss.item()

            all_preds.extend(preds.tolist())
            all_trues.extend(labels.tolist())


        # metrics 
        acc_total = accuracy_score(y_true=all_trues, y_pred=all_preds)
        precision = precision_score(y_true=all_trues, y_pred=all_preds, zero_division=1, average=avg)
        recall = recall_score(y_true=all_trues, y_pred=all_preds, zero_division=1, average=avg)
        f1 = f1_score(y_true=all_trues, y_pred=all_preds, zero_division=1, average=avg)

        avg_eval_loss = eval_loss / (len(val_loader)*batch_size)

        metrics = {
            'accuracy': acc_total, 
            'precision': precision, 
            'recall': recall, 
            'f1': f1, 
            'avg_eval_loss': avg_eval_loss
        }
        wandb.log(metrics)
        print('****Evaluation****')
        print(f'total_accuracy: {acc_total}')

        return acc_total


In [32]:
# training hyperparameters & functions/tools
lr = .001 
num_epochs = 500
run_name = 'Basic CNN test'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')

best_val_acc = 0.0
optimizer = torch.optim.Adam(model.parameters(), lr=lr) #stocastic gradient descent for our optimization algorithm


Using device cuda


In [33]:
# before training, set up wandb for tracking purposes
os.environ["WANDB_API_KEY"] = "5a08d1ebbf0e86ab877a128b98be3c320301b6a0"

run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="agafiabschool-stephen-f-austin-state-university",
    # Set the wandb project where this run will be logged.
    project="Research Project for CSCI-1465",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": lr,
        "architecture": "Basic CNN",
        "dataset": "CrisisMMD",
        "epochs": num_epochs,
    }, name=run_name
)

Train Loss,▁▅▆█▇▃
epoch,▁
Train Loss,0.54165
epoch,1


In [34]:
# time to train
model.to(device)
os.chdir(home)
Path(research_dir, 'models' ).mkdir(parents=True, exist_ok=True)

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}')
    wandb.log({'epoch': epoch+1})

    loss = 0

    for b, batch in tqdm.tqdm(enumerate(train_loader), 
                         total= len(train_loader), desc=f"Processing training data in epoch {epoch+1}"):
        model.train()
        images = batch['image'].to(device)
        labels = batch['label'].to(device)

        model.zero_grad() 
        optimizer.zero_grad()

        # forward pass
        raw_logits = model.forward(images)
        # loss - can use raw logits for this function b/c it applies LogSoftmax 
        loss = nn.CrossEntropyLoss()(raw_logits, labels)

        wandb.log({'Train Loss': loss.item()})

        # backprop!
        loss.backward()

        optimizer.step()

        if (b+1) % 10 == 0:
            print(f'batch: {b+1} ; loss: {loss.item()}')
    
    # each epoch, run validation
    acc = dev(model=model, val_loader=val_loader)
    
    if acc > best_val_acc:
        best_val_acc = acc
        torch.save(model, Path(research_dir / 'models', {run_name}))
    




Epoch 1


Processing validation data: 100%|██████████| 1/1 [00:00<00:00, 14.35it/s]


AttributeError: 'list' object has no attribute 'cpu'

In [ ]:
loaded_model_dict = torch.load(PATH)

In [ ]:
loaded_model_dict.keys()

In [ ]:
loaded_model = ConvNet().load_state_dict(state_dict=loaded_model_dict)